In [1]:
import os

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
cache_dir = "/root/autodl-tmp"



from transformers import AutoTokenizer, AutoModelForCausalLM

import torch


In [2]:

model_name = "Qwen/Qwen1.5-7B-Chat"

model_cache_dir = "Qwen1.5-7B-Chat"


tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=os.path.join(cache_dir, model_cache_dir), trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                             device_map="auto",
                                             torch_dtype=torch.bfloat16,
                                             cache_dir=os.path.join(cache_dir, model_cache_dir))





Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:


from IPython.display import display


tokenizer.apply_chat_template??
help(tokenizer)
tokenizer.chat_template



Help on Qwen2TokenizerFast in module transformers.models.qwen2.tokenization_qwen2_fast object:

class Qwen2TokenizerFast(transformers.tokenization_utils_fast.PreTrainedTokenizerFast)
 |  Qwen2TokenizerFast(vocab_file=None, merges_file=None, tokenizer_file=None, unk_token='<|endoftext|>', bos_token=None, eos_token='<|endoftext|>', pad_token='<|endoftext|>', **kwargs)
 |  
 |  Construct a "fast" Qwen2 tokenizer (backed by HuggingFace's *tokenizers* library). Based on byte-level
 |  Byte-Pair-Encoding.
 |  
 |  Same with GPT2Tokenizer, this tokenizer has been trained to treat spaces like parts of the tokens so a word will
 |  be encoded differently whether it is at the beginning of the sentence (without space) or not:
 |  
 |  ```python
 |  >>> from transformers import Qwen2TokenizerFast
 |  
 |  >>> tokenizer = Qwen2TokenizerFast.from_pretrained("Qwen/Qwen-tokenizer")
 |  >>> tokenizer("Hello world")["input_ids"]
 |  [9707, 1879]
 |  
 |  >>> tokenizer(" Hello world")["input_ids"]
 |  [2

"{% for message in messages %}{% if loop.first and messages[0]['role'] != 'system' %}{{ '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n' }}{% endif %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"

Signature:
tokenizer.apply_chat_template(
    conversation: Union[List[Dict[str, str]], List[List[Dict[str, str]]], ForwardRef('Conversation')],
    chat_template: Optional[str] = None,
    add_generation_prompt: bool = False,
    tokenize: bool = True,
    padding: bool = False,
    truncation: bool = False,
    max_length: Optional[int] = None,
    return_tensors: Union[str, transformers.utils.generic.TensorType, NoneType] = None,
    return_dict: bool = False,
    tokenizer_kwargs: Optional[Dict[str, Any]] = None,
    **kwargs,
) -> Union[str, List[int], List[str], List[List[int]], transformers.tokenization_utils_base.BatchEncoding]
Source:   
    def apply_chat_template(
        self,
        conversation: Union[List[Dict[str, str]], List[List[Dict[str, str]]], "Conversation"],
        chat_template: Optional[str] = None,
        add_generation_prompt: bool = False,
        tokenize: bool = True,
        padding: bool = False,
        truncation: bool = False,
        max_length: O

In [4]:

target_module = "q_proj"

for name, paramter in model.named_parameters():
    if target_module in name:
        print(name)



model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.q_proj.bias
model.layers.1.self_attn.q_proj.weight
model.layers.1.self_attn.q_proj.bias
model.layers.2.self_attn.q_proj.weight
model.layers.2.self_attn.q_proj.bias
model.layers.3.self_attn.q_proj.weight
model.layers.3.self_attn.q_proj.bias
model.layers.4.self_attn.q_proj.weight
model.layers.4.self_attn.q_proj.bias
model.layers.5.self_attn.q_proj.weight
model.layers.5.self_attn.q_proj.bias
model.layers.6.self_attn.q_proj.weight
model.layers.6.self_attn.q_proj.bias
model.layers.7.self_attn.q_proj.weight
model.layers.7.self_attn.q_proj.bias
model.layers.8.self_attn.q_proj.weight
model.layers.8.self_attn.q_proj.bias
model.layers.9.self_attn.q_proj.weight
model.layers.9.self_attn.q_proj.bias
model.layers.10.self_attn.q_proj.weight
model.layers.10.self_attn.q_proj.bias
model.layers.11.self_attn.q_proj.weight
model.layers.11.self_attn.q_proj.bias
model.layers.12.self_attn.q_proj.weight
model.layers.12.self_attn.q_proj.bias
model.

In [5]:


from datasets import Dataset

ds = Dataset.load_from_disk("./alpaca_data_zh/")

ds[:1]


{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。'],
 'input': [''],
 'instruction': ['保持健康的三个提示。']}

In [6]:
"""
由于数据集是单轮对话，所以只需要做简单的处理即可
"""

def process_func(example):

    max_length = 384

    input_ids, attention_mask, labels = [], [], []
    example_prompt = example["instruction"] + "\n" + example["input"].strip()
    messages = [{"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": example_prompt}]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    instruction = tokenizer(text, add_special_tokens=False)
    
    response = tokenizer(example["output"], add_special_tokens=False)
    # print("11:", type(instruction))
    # print("22:", type(response))
    # try:
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1] # eos token也是需要关注的
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]

    if len(input_ids) > max_length:
        input_ids = input_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        labels = labels[:max_length]
    
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}
    

tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)


In [7]:
# lora 微调

from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(task_type=TaskType.CAUSAL_LM, 
                   target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
                   inference_mode=False,
                   r=8,   # lora的秩
                   lora_alpha=32,  # lora alpha, lora的缩放，缩放的原则是alpha / r， 目前这个参数是缩放4倍（增大lora的影响和贡献度）
                   lora_dropout=0.1)

print("lora config: \n", config)


from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

args = TrainingArguments(output_dir="/root/autodl-tmp/qwen1.5-7b-chat-lora-fine-tuning",
                         per_device_train_batch_size=2, 
                         gradient_accumulation_steps=2,
                        learning_rate=5e-5,
                        logging_steps=20,
                        num_train_epochs=1,
                        save_steps=50,save_total_limit=1,
                        # fp16=16   # 启动混合精度训练
                        )


peft_model = get_peft_model(model, config)

trainer = Trainer(model=peft_model,
                 args=args,train_dataset=tokenized_ds, data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer))


trainer.train()


lora config: 
 LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'q_proj', 'down_proj', 'v_proj', 'k_proj', 'gate_proj', 'o_proj', 'up_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)


Step,Training Loss
20,2.312100
40,1.675500
60,1.597100
80,1.615600
100,1.568000
120,1.652400
140,1.471300
160,1.581200
180,1.427300
200,1.541300


/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecate

TrainOutput(global_step=6714, training_loss=1.4903508204237705, metrics={'train_runtime': 3247.0713, 'train_samples_per_second': 8.271, 'train_steps_per_second': 2.068, 'total_flos': 1.2960755261015654e+17, 'train_loss': 1.4903508204237705, 'epoch': 0.9999255342914588})

In [8]:
# tokenizer.pad_token, tokenizer.pad_token_id, tokenizer.eos_token, tokenizer.eos_token_id, tokenizer.decode([1])

In [9]:
"""
下面是为了测试qwen1.5-7b-chat模型在tokenizer下如何按照模型的方式去组织输入内容，并观察区别，实际有两种方式：

前提：目前测试的数据是单轮对话数据

1、按照官方文档进行组织模型的输入（按照模型的chat_template进行组织输入），思想：按照多轮对话的方式组织单轮对话
2、

"""
prompt = "我今天在家学习和工作"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

example = ds[0]
"""
example: 
{'output': '以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。', 'input': '', 'instruction': '保持健康的三个提示。'}
"""


# 第一种方式：按照多轮对话的方式来组织单轮对话
example_prompt = example["instruction"] + "\n" + example["input"].strip()

print("example_prompt: ", example_prompt)
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": example_prompt}
]

text = tokenizer.apply_chat_template(messages, 
                                     tokenize=False,
                                     add_generation_prompt=True)


print(text)


# 第二种方式：将第一种方式获取得到的字符串chat_template来组织， 输入的prompt部分用占位符代替
text_chat_template = text.replace("保持健康的三个提示。", "{}")

print(text_chat_template)

print(text_chat_template.format(example_prompt))


example_prompt:  保持健康的三个提示。

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
保持健康的三个提示。
<|im_end|>
<|im_start|>assistant

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
{}
<|im_end|>
<|im_start|>assistant

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
保持健康的三个提示。

<|im_end|>
<|im_start|>assistant

